<a href="https://colab.research.google.com/github/shinmiura/diveintocode-ml/blob/master/%E3%82%A2%E3%83%B3%E3%82%B5%E3%83%B3%E3%83%96%E3%83%AB%E5%AD%A6%E7%BF%92.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.このSprintについて

**Sprintの目的**

アンサンブル学習について理解する


**どのように学ぶか**

スクラッチでアンサンブル学習の各種手法を実装していきます。



# 2.アンサンブル学習

3種類のアンサンブル学習をスクラッチ実装していきます。そして、それぞれの効果を小さめのデータセットで確認します。


・ブレンディング

・バギング

・スタッキング

**小さなデータセットの用意**

以前も利用した回帰のデータセットを用意します。


House Prices: Advanced Regression Techniques


この中のtrain.csvをダウンロードし、目的変数としてSalePrice、説明変数として、GrLivAreaとYearBuiltを使います。


train.csvを学習データ8割、検証データ2割に分割してください。



**scikit-learn**

単一のモデルはスクラッチ実装ではなく、scikit-learnなどのライブラリの使用を推奨します。


sklearn.linear_model.LinearRegression — scikit-learn 0.21.3 documentation


sklearn.svm.SVR — scikit-learn 0.21.3 documentation


sklearn.tree.DecisionTreeRegressor — scikit-learn 0.21.3 documentation

# 3.ブレンディング

【問題1】ブレンディングのスクラッチ実装

ブレンディング をスクラッチ実装し、単一モデルより精度があがる例を **最低3つ** 示してください。精度があがるとは、検証データに対する平均二乗誤差（MSE）が小さくなることを指します。

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, SGDRegressor
from sklearn.linear_model import PassiveAggressiveRegressor, ARDRegression, RidgeCV
from sklearn.linear_model import TheilSenRegressor, RANSACRegressor, HuberRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR, LinearSVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import log_loss
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingRegressor

In [2]:
# データの読み込み
data = pd.read_csv('/content/drive/MyDrive/DIC/train.csv')
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,...,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,...,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,...,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,...,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,...,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
# 特徴量２つ(最初の課題と同じものを選択)、ターゲット変数のみに絞ったデータを作成
min_data = data[['GrLivArea' ,'YearBuilt' ,'SalePrice']]
min_data.head()

,GrLivArea,YearBuilt,SalePrice
0,1710,2003,208500
1,1262,1976,181500
2,1786,2001,223500
3,1717,1915,140000
4,2198,2000,250000


In [4]:
X = min_data.iloc[:, 0:2]
X.head()

,GrLivArea,YearBuilt
0,1710,2003
1,1262,1976
2,1786,2001
3,1717,1915
4,2198,2000


In [5]:
y = min_data.iloc[:, -1]
y.head()

0    208500
1    181500
2    223500
3    140000
4    250000
Name: SalePrice, dtype: int64

In [6]:
# 訓練データとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = None)

In [7]:
# モデルの設定(初期化して辞書に格納)
models = {'線形回帰' : LinearRegression(), 'SVM' : SVR(), '決定木' : DecisionTreeRegressor()}

〈問題１解答前半〉

単一モデルの場合の精度をまず以下のとおり算出した。

In [8]:
for model_name, model in models.items():
  # X_trainとy_trainを使ってモデルの学習
  model.fit(X_train, y_train)
  # 学習済モデルとX_testを使って予測
  y_pred = model.predict(X_test)
  # y_testと予測結果の答え合わせ
  mse = mean_squared_error(y_test, y_pred)
  print('{}モデルのRMSE : {}'.format(model_name, np.sqrt(mse)))

線形回帰モデルのRMSE : 41990.699746363214
SVMモデルのRMSE : 79208.22345317087
決定木モデルのRMSE : 53621.55996943175


ブレンディングとは

ブレンディングとは、N個の多様なモデルを独立して学習させ、推定結果を重み付けした上で足し合わせる方法です。最も単純には平均をとります。多様なモデルとは、以下のような条件を変化させることで作り出すものです。


・手法（例：線形回帰、SVM、決定木、ニューラルネットワークなど）

・ハイパーパラメータ（例：SVMのカーネルの種類、重みの初期値など）

・入力データの前処理の仕方（例：標準化、対数変換、PCAなど）

重要なのはそれぞれのモデルが大きく異なることです。


回帰問題でのブレンディングは非常に単純であるため、scikit-learnには用意されていません。



《補足》


分類問題の場合は、多数決を行います。回帰問題に比べると複雑なため、scikit-learnにはVotingClassifierが用意されています。


sklearn.ensemble.VotingClassifier — scikit-learn 0.21.3 documentation

〈問題１解答後半〉

単一モデルよりも精度が上がる例を以下のとおり検証した。
下記１〜３のいずれのアプローチも単一モデルに比して精度が下がっていることを確認できた。

アプローチ１：複数モデルの予測値の平均をとる

アプローチ２：学習時の乱数シードを変えて平均をとる

アプローチ３：同じモデル×複数で各ハイパーパラメータを変えて平均をとる

In [9]:
# アプローチ１：複数モデルの予測値の平均をとる
lm = LinearRegression()
sv = SVR()
dt = DecisionTreeRegressor()
#  複数の(ここでは３つに設定)モデルを個別に学習
model1 = lm.fit(X_train, y_train)
model2 = sv.fit(X_train, y_train)
model3 = dt.fit(X_train, y_train)
#  複数の(ここでは３つに設定)学習済モデルとX_testを使って予測
pred1 = lm.predict(X_test)
pred2 = sv.predict(X_test)
pred3 = dt.predict(X_test)

app1_pred = np.mean([pred1, pred2, pred3], axis = 0)

# y_testと予測結果の答え合わせ
app1_mse = mean_squared_error(y_test, app1_pred)
print('3モデルの平均のRMSE : {}'.format(np.sqrt(app1_mse)))

3モデルの平均のRMSE : 46366.60188233441


In [10]:
# アプローチ２：学習時の乱数シードを変えて平均をとる
# 訓練データとテストデータに分割する際に乱数シードを変える
X2_train, X2_test, y2_train, y2_test = train_test_split(X, y, test_size = 0.3, random_state = 1234)
lm = LinearRegression()
sv = SVR()
dt = DecisionTreeRegressor()
#  複数の(ここでは３つに設定)モデルを個別に学習
model12 = lm.fit(X2_train, y2_train)
model22 = sv.fit(X2_train, y2_train)
model32 = dt.fit(X2_train, y2_train)
#  複数の(ここでは３つに設定)学習済モデルとX_testを使って予測
pred12 = lm.predict(X2_test)
pred22 = sv.predict(X2_test)
pred32 = dt.predict(X2_test)

app2_pred = np.mean([pred12, pred22, pred32], axis = 0)

# y_testと予測結果の答え合わせ
app2_mse = mean_squared_error(y2_test, app2_pred)
print('学習時の乱数シードを変えた場合の平均のRMSE : {}'.format(np.sqrt(app2_mse)))

学習時の乱数シードを変えた場合の平均のRMSE : 42301.977651964655


In [11]:
# アプローチ３：同じモデル×複数で各ハイパーパラメータを変えて平均をとる
dt1 = DecisionTreeRegressor(max_depth = 5)
dt2 = DecisionTreeRegressor(max_depth = 10)
dt3 = DecisionTreeRegressor(max_depth = 15)
#  複数の(ここでは３つに設定)モデルを個別に学習
model1 = dt1.fit(X_train, y_train)
model2 = dt2.fit(X_train, y_train)
model3 = dt3.fit(X_train, y_train)
#  複数の(ここでは３つに設定)学習済モデルとX_testを使って予測
pred1 = dt1.predict(X_test)
pred2 = dt2.predict(X_test)
pred3 = dt3.predict(X_test)

app3_pred = np.mean([pred1, pred2, pred3], axis = 0)

# y_testと予測結果の答え合わせ
app3_mse = mean_squared_error(y_test, app3_pred)
print('同一モデルで各ハイパーパラメータを変えた場合の平均のRMSE : {}'.format(np.sqrt(app3_mse)))

同一モデルで各ハイパーパラメータを変えた場合の平均のRMSE : 39407.168731246325


# 4.バギング

【問題2】バギングのスクラッチ実装

バギング をスクラッチ実装し、単一モデルより精度があがる例を 最低1つ 示してください。

**バギングとは**

バギングは入力データの選び方を多様化する方法です。訓練データから重複を許した上でランダムに抜き出すことで、N種類のサブセット（ ブートストラップサンプル ）を作り出します。それらによってモデルをN個学習し、推定結果の平均をとります。ブレンディングと異なり、それぞれの重み付けを変えることはありません。


sklearn.model_selection.train_test_split — scikit-learn 0.21.3 documentation


推定結果の平均をとる部分はブレンディングと同様の実装になります。

〈問題2解答前半〉

単一モデルの場合の精度をまず以下のとおり算出した。

In [12]:
# 単一モデルを初期化
dt = DecisionTreeRegressor()
# X_trainとy_trainを使ってモデルの学習
model = dt.fit(X_train, y_train)
# 学習済モデルとX_testを使って予測
pred = model.predict(X_test)
# y_testと予測結果の答え合わせ
mse = mean_squared_error(y_test, pred)
print('単一モデルのRMSE : {}'.format(np.sqrt(mse)))

単一モデルのRMSE : 53240.28069810585


〈問題２解答後半〉

単一モデルよりも精度が上がる例を以下のとおり検証した。以下のとおりバギングを実施した結果、精度が上がっていることを確認できた。

In [13]:
# ブートストラップする回数を決定
n_boot = 100

In [14]:
# バギングによって取り出した各データに対して作成したモデルをリストに格納
models = []

In [15]:
# ブートストラップをfor文で行う
for i in range(n_boot):
  # 訓練データをさらにランダムに復元抽出サンプル(重複を許して抽出するため引数にshuffle = Trueとする)
  X_bagging, _X, y_bagging, _y = train_test_split(X_train, y_train, train_size = 0.3, shuffle = True)
  # 弱学習器を構築
  dt = DecisionTreeRegressor()
  # 学習
  dt.fit(X_bagging, y_bagging)
  # モデルを集約
  models.append(dt)

In [16]:
# 予測値の初期値を設定
prediction = np.zeros(len(X_test))
# 集約したモデルを集計し、モデル数(= ブートストラップする回数)で除して平均値を算定
for model in models:
  pred_value = model.predict(X_test)
  prediction += pred_value

bag_mean = prediction / n_boot 
# y_testと予測結果の答え合わせ
bag_mse = mean_squared_error(y_test, bag_mean)
print('決定木でバギングを行った場合の平均のRMSE : {}'.format(np.sqrt(bag_mse)))

決定木でバギングを行った場合の平均のRMSE : 35527.187579245525


# 5.スタッキング

【問題3】スタッキングのスクラッチ実装

スタッキング をスクラッチ実装し、単一モデルより精度があがる例を **最低1つ** 示してください。

**スタッキングとは**

スタッキングの手順は以下の通りです。最低限ステージ0とステージ1があればスタッキングは成立するため、それを実装してください。まずは $K_0=3, M_0=2$ 程度にします。

《学習時》


（ステージ $0$ ）


・訓練データを $K_0$ 個に分割する。

・分割した内の $(K_0 - 1)$ 個をまとめて訓練データ、残り $1$ 個を推定用データとする組み合わせが $K_0$ 個作れる。

・あるモデルのインスタンスを $K_0$ 個用意し、異なる訓練データを使い学習する。

・それぞれの学習済みモデルに対して、使っていない残り $1$ 個の推定用データを入力し、推定値を得る。（これをブレンドデータと呼ぶ）

・さらに、異なるモデルのインスタンスも $K_0$ 個用意し、同様のことを行う。モデルが $M_0$ 個あれば、 $M_0$ 個のブレンドデータが得られる。

（ステージ $n$ ）


・ステージ $n-1$ のブレンドデータを$M_{n-1}$ 次元の特徴量を持つ訓練データと考え、 $K_n$ 個に分割する。以下同様である。

（ステージ $N$ ）＊最後のステージ


・ステージ $N-1$ の $M_{N-1}$ 個のブレンドデータを$M_{N-1}$ 次元の特徴量の入力として、1種類のモデルの学習を行う。これが最終的な推定を行うモデルとなる。

《推定時》


（ステージ $0$ ）


・テストデータを $K_0×M_0$ 個の学習済みモデルに入力し、$K_0×M_0$ 個の推定値を得る。これを $K_0$ の軸で平均値を求め $M_0$ 次元の特徴量を持つデータを得る。（ブレンドテストと呼ぶ）

（ステージ $n$ ）


・ステージ $n-1$ で得たブレンドテストを $K_n×M_n$ 個の学習済みモデルに入力し、$K_n×M_n$ 個の推定値を得る。これを $K_n$ の軸で平均値を求め $M_0$ 次元の特徴量を持つデータを得る。（ブレンドテストと呼ぶ）

（ステージ $N$ ）＊最後のステージ


・ステージ $N-1$ で得たブレンドテストを学習済みモデルに入力し、推定値を得る。


In [21]:
class Stacking():
  """
  スタッキングの実装
  """
  def __init__(self, max_depth, splits, models):
    """
    コンストラクタ
    Parameters
    --------------
    max_depth : スタッキングする層
    splits : 分割するsplits数
    models : 適用するモデル
    learned_models :　学習したモデル
    """
    self.max_depth = max_depth
    self.n_splits = splits
    self.models = models
    self.learned_models = [] 
  
  def calc_blend(self, X, y, model):
    """
    ブレンドデータを得る
    Parameters
    --------------
    model : モデル
    X : 特徴量
    y : 目的変数
    """
    # ブレンドデータの初期化
    self.y_blend = np.zeros(len(X))

    # 学習データをクロスバリデーションのfoldに分ける(Diverの記載に基づいてfoldを1から3とする)
    kf = KFold(n_splits = self.n_splits, shuffle = False, random_state = 1234)
    
    # fold分繰り返す
    for tr_idx, va_idx in kf.split(X):
      # データを分割
      X_train, X_val = X[tr_idx], X[va_idx]
      y_train, y_val = y[tr_idx], y[va_idx]
      
      y_train = y_train.ravel()
      y_valid = y_valid.ravel()

      # 訓練データを用いた学習モデルの作成
      regr = model
      regr.fit(X_train, y_train)
      self.fit_models.append(regr)
      # ブレンドデータの作成
      self.y_blend[valid_index] = regr.predict(X_valid)

  def fit(self, X, y, depth):
    """
    2層目以降の学習
    Parameters
    --------------
    X : 特徴量
    y : 目的変数
    depth : 何層目のスタッキングか
    """
    # 現在の何層目にいるかをインスタンス化
    self.depth = depth

    # 最深層に達したら、最終層のモデルを学習し(当該モデルの予測値を最終的な予測値とする)処理をいったん終了させる
    if self.depth == self.max_depth:
      # 最深層のモデルを取得
      self.model = self.models[self.depth]
      # 最深層のモデルを学習
      self.model.fit(X, y)
      return
    
    # 各層におけるモデルを取得
    models = self.models[self.depth]
    self.blend = np.zeros([len(X), len(models)])

    #　各層ごとに全てのモデルで学習・予測
    for i, mdl in enumerate(models):
      self.calc_blend(X, y, mdl)
      self.blend[:, i] = self.y_blend
    
    # (注)授業のコード解説を見たままメモして書いているので意味が理解できていないところ！！
    self.bld = Stacking(self.max_depth, self.n_splits, self.models)
    # １つ前の層のモデルの予測値を特徴量として使って学習
    self.bld.fit_(blend_y, y, depth + 1)
    # 次の層へ移る
    self.depth += 1

  def predict(self, X):
    """
    2層目以降の予測
    Parameters
    --------------
    X : 特徴量
    y : 目的変数
    depth_now : 何層目のスタッキングか
    """

    # 最深層に達したら、最終層のモデルの予測値を最終的な予測値として処理
    if self.depth == self.max_depth:
      prediction = self.model.predict(X)
      return prediction
    
    # 最大深さに達していない場合には、再帰的に呼び出す
    else:
      # 予測値の初期化
      self.prediction = np.zeros(len(X))
      # 次の層の特徴量として渡すブレンドデータを作成
      self.y_next = np.zeros([len(X), len(self.models[self.depth])])
      # どのモデルかを把握(初期値は0)
      model_count = 0
      for model in self.learned_models:
        self.prediction += model.predict(X)
        model_count += 1
        if model_count % self.n_splits == 0:
          # 取得してきた予測値の平均をとる
          self.prediction = self.prediction / self.n_splits
          # 上記の平均値を次の層の特徴量として使えるようにブレンドデータに格納
          self.y_next[:, int(model_count / self.n_splits) -1] = self.prediction
          # 次のモデルに影響してしまわないようにこの時点(次のモデルに移る前に)でも予測値の初期化が必要
          self.prediction = np.zeros(len(X))
      # 次の層の予測関数
      prediction = self.stack.predict(self.blending)
    return prediction

In [22]:
# データを分割し、Numpyに変換
X_train, X_test, y_train, y_test = train_test_split(X.to_numpy(), y.to_numpy(), test_size = 0.3, random_state = None)

In [23]:
# あるモデルのインスタンスを $K_0$ 個用意
models = {
    0 : [LinearRegression(), SVR(), DecisionTreeRegressor()],
    1 : [ARDRegression(), HuberRegressor(), RandomForestRegressor()],
    2 : [ARDRegression(), RandomForestRegressor(), DecisionTreeRegressor()],
    3 : LinearRegression()
}

# 関数の呼び出し
stack = Stacking(max_depth = 3, splits = 5, models = models)
stack.fit(X_train, y_train, 0)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


UnboundLocalError: ignored

In [ ]:
prediction = stack.predict(X_test)

In [ ]:
# 指標
mse = mean_squared_error(y_test, prediction)
print('MSE : {:.3f}'.format(mse))